In [13]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
from datetime import datetime

In [2]:
#[시작]###########################################################
#디렉토리 만들기 - 완료
# path = r'\\192.168.0.75\솔림헬프'
# os.chdir(path)
# docu_list = ["1.원인서류", "2.양도통지서", "3.집행권원", "4.강제집행", "5.등초본", "6.개인회생", "7.신용회복", "8.파산", "9.재산조사", "기타"]
# sell_list = ["전북-01", "DNP-01", "DNP-02", "메리츠캐_01", "KNS", "DNP-03", "한울가람", "케이에스", \
#     "유안타저-01", "KB카드-01", "베리타스-01", "BNK-01", "스마트저축-01", "우담-01", "고려저축-01", \
#         "롯데카드-01", "헤럴드", "KCR", "아인스", "하나캐-02", "베델-01", "롯데카드-03", "하나캐-01", \
#             "더키움", "롯데카드-02", "JT캐-02", "흥국화재-01", "롯데카드-04", "산은캐-01", "대산", "JT캐-01"]

# for dir_ in docu_list :
#     for sell_ in sell_list :
#         os.makedirs(dir_+"/"+sell_, exist_ok=True)

#[끝]###########################################################

In [39]:
path = r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)'
os.chdir(path)
file_list = [f.name for f in os.scandir(path) if f.is_file()]

In [43]:
for f in file_list :
    if f != "Thumbs.db" :
        if len(re.findall("_", f)) < 2 :
            ff.re_name(join(path,f), join(path,"파일명 수정 필요", f))

In [38]:
f = "___"
re.find("_", f)

AttributeError: module 're' has no attribute 'find'

### 나스에 파일 올리기

##### 파일명 체크하기

In [37]:
path = r'C:\Users\jbmyo\Desktop\test'
os.chdir(path)
file_list = [f.name for f in os.scandir(path) if f.is_file()]
cnt = 0
filiname_error = []

docu_kind = r'원인서류|양도통지서[\s]?[\d]?[차]?|집행권원[\s]?[재]?[도]?|집행권원 재도|집행권원재도|강제집행|등본|초본|외국인증명|개인회생|신용회복|파산|재산조사|기타'

p_basic = re.compile(r'([\d]{8})([\D]+)('+docu_kind+')')
p_event = re.compile(r"[\D]([\d]{4}[ㄱ-ㅎ가-힣]{1,3}[0-9]+)")


<DirEntry 'Thumbs.db'>
<DirEntry '진현철 721013-1244712(초본).pdf'>


In [ ]:
for f in file_list :
    try :
        stem = os.path.splitext(f)[0]
        ext = os.path.splitext(f)[1]
        basic = p_basic.match(stem)
        
        key = basic.group(1)
        name = basic.group(2).strip()
        docu = basic.group(3)
        
        if p_event.search(stem) :
            temp_event = p_event.search(stem)
            event = temp_event.group()
            extra = stem[temp_event.end():].strip().replace(" ", "_")
        else :
            extra = stem[basic.end():].strip().replace(" ", "_")

        if extra :
            name_items = [key, name, docu, event, extra]
        else :
            name_items = [key, name, docu, event]

        new_name = "_".join(name_items)+ext
        new_name = re.sub("[_]{2,}", "_", new_name)
        
        shutil.move(f, join(path, "최종변경 완료",new_name))
        cnt += 1
    except :
        print("===================================")
        print(datetime.today().strftime("%H:%M:%S"))
        print(f)
        print(traceback.format_exc())
        filiname_error.append(f)


print(cnt, "개의 파일 이동 완료")

##### "솔림헬프"로 파일 업로드

In [19]:
#[시작]파일업로드###########################################################
# 참조 df 불러오기 ~~~~ ":" xlsx파일에서 미리 제거해둬야 ~~~~
df_c = pd.read_excel(r'C:\Users\SL\Desktop\채무자조회.xlsx')
df_c.tail()

,채무자키,매각사구분,채권구분,성명,주민번호인,담당자,채무상태,성별,연령,관리자기타,보증인성명,보증인상태,변경전성명인,변경전주민번호인,보증인수
67722,20459682,KB카드-01,NPL,YOON HYUN WOOK,671224-5600173,환매,환매,남자,54,9013302712,NaN,NaN,NaN,NaN,0
67723,20459636,KB카드-01,NPL,YUTAIJI,830211-5140013,환매,환매,남자,39,9013302093,NaN,NaN,NaN,NaN,0
67724,20447783,KB카드-01,NPL,ZAHER KAREEM ATIA,900803-5100144,정해진,정상,남자,32,9013302590,NaN,NaN,NaN,NaN,0
67725,20459338,KB카드-01,NPL,ZHANG FANJING,871111-6340392,환매,환매,여자,34,9013202025,NaN,NaN,NaN,NaN,0
67726,20459630,KB카드-01,NPL,ZONG WEI,580805-6101696,환매,환매,여자,64,9013302001,NaN,NaN,NaN,NaN,0


In [20]:
# 채무상태와 매각사구분만 뺀 dict
dict_sell ={}
for ind, col in df_c.iterrows():
    dict_sell[str(col.채무자키)] = [col.매각사구분, col.채무상태]

print(len(dict_sell))

67727


In [44]:
# 경로 정리
path = r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)'
os.chdir(path)
file_list = [f.name for f in os.scandir(path) if f.is_file()]

f_d = r'\\192.168.0.75\솔림헬프'
out = r'\\192.168.0.75\삭제예정파일\관리제외'
# 기본 변수
p_out = re.compile('개인회생\(면책\)|파산\(면책\)|환매|매각|종결')
p_key = re.compile(r"([\d]{8})[\D]") #match

error = []
log = []
# 1) depth1 문서종류 매칭
docu_match = {"원인서류" : "1.원인서류", "양도통지서":"2.양도통지서", "집행권원":"3.집행권원", "강제집행":"4.강제집행", "등본":"5.등초본", "초본":"5.등초본", "외국인":"5.등초본", "개인회생":"6.개인회생", "신용회복":"7.신용회복", "파산":"8.파산", "재산조사":"9.재산조사", "기타":"기타"}

In [45]:
for f in tqdm(file_list) : 
    
    if (f != "Thumbs.db") :
        
        try :
            depth1 = "" #문서종류
            depth2 = "" #매각사구분
            depth3 = f[:8] #다 확인한 뒤니까 그냥 이렇게 해도 돼
        
            # 관리제외라면
            if p_out.match(dict_sell[depth3][1]) :
                out_dir = join(out, depth3)
                if not os.path.exists(out_dir) :
                    os.mkdir(out_dir) 
                shutil.move(join(path,f), join(out_dir, f))
            
            # 관리파일이면
            else :
            
                # depth1
                for key, value in docu_match.items() : 
                    if re.search(key, f) :
                        depth1 = value
                        continue
                    else :
                        pass

                # depth2
                depth2 = dict_sell[depth3][0]
                
                # 파일이동을 위한 준비(도착지 디렉토리 및 파일명 작성)
                dst_dir = os.path.join(f_d, depth1, depth2, depth3)
            
                if not os.path.exists(dst_dir) :
                    os.mkdir(dst_dir) # depth2까지는 만들어놨으니 s안 붙여도 됨
                
                #파일이동
                log.append(ff.re_name(join(path, f), join(dst_dir, f)))
        
        except :
            print("===================================")
            print(datetime.today().strftime("%H:%M:%S"))
            print(f)
            print(traceback.format_exc())
            error.append(f)
            continue # 반복문 계속 돌아


# 로그 파일 나스에 저장
log_path = r'\\192.168.0.75\스캔파일\스캔파일 log\success'
time = str(datetime.today().strftime("%Y%m%d %H%M%S"))
name = time + ".txt"
index = 1
with open(join(log_path,name), "w") as lf :
    for a, b in log :
        lf.write(str(index))
        lf.write('\t')
        lf.write(a)
        lf.write('\t')
        lf.write(b)
        lf.write('\n')
        index += 1
# 에러 목록 내 컴퓨터에 저장
if len(error) > 0 :
    df = pd.DataFrame(error, columns=["error file"])
    error_path = r'\\192.168.0.75\스캔파일\스캔파일 log\fail'
    error_name = time + ".csv" 
    df.to_csv(join(error_path, error_name))

print(len(log), "개의 파일 업로드 완료", len(error), "개의 요확인 파일 발생")
    

  0%|          | 0/24068 [00:00<?, ?it/s]

06:29:58
20427101_김운기_초본.pdf
Traceback (most recent call last):
  File "c:\Users\sl\anaconda3\lib\shutil.py", line 823, in move
    os.rename(src, real_dst)
OSError: [WinError 17] 시스템은 파일을 다른 디스크 드라이브로 옮길 수 없습니다: '\\\\192.168.0.75\\스캔파일\\새 스캔파일(업로드)\\20427101_김운기_초본.pdf' -> '\\\\192.168.0.75\\솔림헬프\\5.등초본\\더키움\\20427101/20427101_김운기_초본.pdf'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SL\AppData\Local\Temp\ipykernel_10832\505126050.py", line 38, in <cell line: 1>
    log.append(ff.re_name(join(path, f), join(dst_dir, f)))
  File "c:\Users\SL\Desktop\workspace\python\fileNaming\file_function.py", line 69, in re_name
    shutil.move(src, dst_final)
  File "c:\Users\sl\anaconda3\lib\shutil.py", line 843, in move
    copy_function(src, real_dst)
  File "c:\Users\sl\anaconda3\lib\shutil.py", line 444, in copy2
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "c:\Users\sl\anaconda3\lib\shutil.py", line 

In [18]:
print(datetime.today().strftime("%H:%M:%S"))

17:06:34


In [5]:
f = "진현철 721013-1244712(초본).pdf"

for i in range(2) :
    try : 
        depth3 = p_key.match(f).group(1) #채무자키
        
        if not all([depth3]) :
            print("error")
        else :
            print("it's ok")
    except :
        print("except error")
        pass
        

1
except error
1
except error


### 파일 비교 및 차집합

In [ ]:
myPath= r'D:\부산 정리_v2'
nasPath=r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)'

In [ ]:
myFile = [f for f in os.scandir(myPath) if f.is_file()]

In [ ]:
nasFile = [f for f in os.scandir(nasPath) if f.is_file()]

In [ ]:
onlyLocal = list(set(myFile) - set(nasFile))
print(len(onlyLocal))

In [ ]:
onlynas = list(set(myFile) - set(nasFile))
print(len(onlynas))